# Negotiate Offer

With this notebook you can start a Negotiation for an existing offer and thus generate an Agreement.

## Preperation

Here some values are initialized that will be necessary for the other steps.


### Set up

Import the requests module and assign the base url of the Dataspace as a variable.

In [ ]:
from pprint import pprint
import requests

base_url = "https://vision-x-api.base-x-ecosystem.org"

### Fill in Values

Fill in the values for the variables below.

In [ ]:
# Your JWT recevied from Keycloak via vision-x-auth.base-x-ecosystem.org
token = "ey..."
token_header = {"Authorization": f"Bearer {token}"}
# The name of your Connector
connector_name = "my-connector"

## Get Federated Catalog

Here you get the contents of the Federated Catalog and then choose an Offer you want to negotiate for.

Gets and prints out the relevant conents of the Federated Catalog.

In [ ]:
url = f"{base_url}/federated/catalog"

response = requests.get(url)
response.raise_for_status()
print("Got the federated catalog")

federated_catalog = response.json()
for catalog in federated_catalog:
  print(f"Catalog from {catalog['originator']}")
  datasets = catalog["dcat:dataset"]
  if not isinstance(datasets, list):
    datasets = [datasets]
  for dataset in datasets:
    if "name" not in dataset:
      continue
    print(f"    Offer for {dataset['name']}")

Pick out an Offer from the printed ones and fill in the below values for the variables accordingly.

In [ ]:
catalog_from = "some-catalog-from"
offer_for = "some-offer-name"

Gets the necessary data about the chosen Offer from the Federated Catalog in order to start a Negotiation and an eventual Transfer in the next steps.

In [ ]:
for catalog in federated_catalog:
  if catalog["originator"] != catalog_from:
    continue
  datasets = catalog["dcat:dataset"]
  if not isinstance(datasets, list):
    datasets = [datasets]
  for dataset in datasets:
    if not "name" in dataset or dataset["name"] != offer_for:
      continue
    provider_id = catalog["dspace:participantId"]
    originator = catalog["originator"]
    policy = dataset["odrl:hasPolicy"]
    offered_asset_id = dataset["id"]

print("Got necessary values\n")

print(f"ProviderId:  {provider_id}")
print(f"Originator: {originator}")
print(f"Policy: {policy}")
print(f"OfferedAssetId: {offered_asset_id}")

## Initiate Negotiation

Here you start a Negotiation for the Offer chosen in the previous step. If the given conditions are satisfied the Negotiation will succeed and an Agreement will be created.

Starts the Negotiation using the values got from the previous step.

In [ ]:
url = f"{base_url}/connectors/{connector_name}/cp/management/v3/contractnegotiations"
payload = {
  "@context": {
    "odrl": "http://www.w3.org/ns/odrl/2/"
  },
  "counterPartyAddress": originator,
  "protocol": "dataspace-protocol-http",
  "policy": policy[0] | {"odrl:assigner": {"@id": provider_id}, "odrl:target": {"@id": offered_asset_id}}
}

response = requests.post(url, json=payload, headers=token_header)
print(response.content)
response.raise_for_status()

negotiation_id = response.json()["@id"]
print(f"Started Negotiation with ID: {negotiation_id}")

Wait a few seconds.

Confirms that the Negotiation succeeded and get the ID corresponding Agreement which will be needed for the eventual Transfer.

In [ ]:
url = f"{base_url}/connectors/{connector_name}/cp/management/v3/contractnegotiations/{negotiation_id}"

response = requests.get(url, headers=token_header)
response.raise_for_status()
print(f"Negotiation data:\n")
pprint(response.json())

agreement_id = response.json()["contractAgreementId"]

print(f"Originator: {originator}")
print(f"Agreement ID: {agreement_id}")

If everything was successful you should now have gotten the releveant information for starting a Transfer. If you get an error it might be that the Negotiation has not finished yet or was rejected.